# Covid-19 New Case Predictor 
### Using Neural Network
-----------------------------------------------
by Roya Salei

Course CS4200- Artificial Intelligence 

Professor: Dr. Husain Zaidi

In [58]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [102]:
# read data from file
df = pd.read_csv("state_ts.csv")
df.tail(10)

,date,state,newCases,cases,newDeaths,deaths,positiveTests,negativeTests,vaccinationsCompleted,hospitalBeds.capacity,hospitalBeds.currentUsageTotal,hospitalBeds.currentUsageCovid,icuBeds.capacity,icuBeds.currentUsageTotal,icuBeds.currentUsageCovid,mask_mandate,daily_infection_rate,daily_death_rate,newCases_density,newDeaths_density
33027,2021-11-02,WY,367.0,103989.0,69.0,1243.0,81211.0,1021342.0,255199.0,1603.0,818.0,195.0,139.0,74.0,42.0,0,0.636213,0.119615,3.779609,0.710608
33028,2021-11-03,WY,414.0,104403.0,0.0,1243.0,81551.0,1025572.0,255597.0,1748.0,939.0,192.0,139.0,76.0,41.0,0,0.717690,0.000000,4.263646,0.000000
33029,2021-11-04,WY,537.0,104940.0,0.0,1243.0,81819.0,1028372.0,255655.0,1748.0,941.0,179.0,139.0,72.0,36.0,0,0.930916,0.000000,5.530381,0.000000
33030,2021-11-05,WY,378.0,105318.0,0.0,1243.0,82057.0,1030728.0,256237.0,1748.0,923.0,181.0,139.0,76.0,36.0,0,0.655282,0.000000,3.892894,0.000000
33031,2021-11-06,WY,0.0,105318.0,0.0,1243.0,82202.0,1032878.0,256695.0,1747.0,881.0,169.0,137.0,79.0,37.0,0,0.000000,0.000000,0.000000,0.000000
33032,2021-11-07,WY,0.0,105318.0,0.0,1243.0,82344.0,1033805.0,256745.0,1747.0,881.0,175.0,137.0,78.0,41.0,0,0.000000,0.000000,0.000000,0.000000
33033,2021-11-08,WY,672.0,105990.0,0.0,1243.0,82367.0,1034107.0,256772.0,1602.0,806.0,185.0,139.0,74.0,41.0,0,1.164946,0.000000,6.920700,0.000000
33034,2021-11-09,WY,297.0,106287.0,55.0,1298.0,NaN,NaN,257742.0,1603.0,831.0,181.0,137.0,81.0,39.0,0,0.514864,0.095345,3.058702,0.566426
33035,2021-11-10,WY,411.0,106698.0,0.0,1298.0,NaN,NaN,258081.0,1748.0,963.0,174.0,136.0,77.0,36.0,0,0.712489,0.000000,4.232750,0.000000
33036,2021-11-11,WY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1748.0,963.0,174.0,136.0,77.0,36.0,0,NaN,NaN,NaN,NaN


In [103]:
#Drop rows that has any NaN data
df = df.dropna()
df = df.reset_index()
df.tail(10)

,index,date,state,newCases,cases,newDeaths,deaths,positiveTests,negativeTests,vaccinationsCompleted,...,hospitalBeds.currentUsageTotal,hospitalBeds.currentUsageCovid,icuBeds.capacity,icuBeds.currentUsageTotal,icuBeds.currentUsageCovid,mask_mandate,daily_infection_rate,daily_death_rate,newCases_density,newDeaths_density
14532,33024,2021-10-30,WY,0.0,102926.0,0.0,1174.0,80690.0,1017147.0,253968.0,...,892.0,193.0,136.0,79.0,47.0,0,0.000000,0.000000,0.000000,0.000000
14533,33025,2021-10-31,WY,0.0,102926.0,0.0,1174.0,80820.0,1017853.0,254028.0,...,864.0,188.0,137.0,76.0,43.0,0,0.000000,0.000000,0.000000,0.000000
14534,33026,2021-11-01,WY,696.0,103622.0,0.0,1174.0,81032.0,1019225.0,254047.0,...,782.0,195.0,138.0,71.0,45.0,0,1.206551,0.000000,7.167868,0.000000
14535,33027,2021-11-02,WY,367.0,103989.0,69.0,1243.0,81211.0,1021342.0,255199.0,...,818.0,195.0,139.0,74.0,42.0,0,0.636213,0.119615,3.779609,0.710608
14536,33028,2021-11-03,WY,414.0,104403.0,0.0,1243.0,81551.0,1025572.0,255597.0,...,939.0,192.0,139.0,76.0,41.0,0,0.717690,0.000000,4.263646,0.000000
14537,33029,2021-11-04,WY,537.0,104940.0,0.0,1243.0,81819.0,1028372.0,255655.0,...,941.0,179.0,139.0,72.0,36.0,0,0.930916,0.000000,5.530381,0.000000
14538,33030,2021-11-05,WY,378.0,105318.0,0.0,1243.0,82057.0,1030728.0,256237.0,...,923.0,181.0,139.0,76.0,36.0,0,0.655282,0.000000,3.892894,0.000000
14539,33031,2021-11-06,WY,0.0,105318.0,0.0,1243.0,82202.0,1032878.0,256695.0,...,881.0,169.0,137.0,79.0,37.0,0,0.000000,0.000000,0.000000,0.000000
14540,33032,2021-11-07,WY,0.0,105318.0,0.0,1243.0,82344.0,1033805.0,256745.0,...,881.0,175.0,137.0,78.0,41.0,0,0.000000,0.000000,0.000000,0.000000
14541,33033,2021-11-08,WY,672.0,105990.0,0.0,1243.0,82367.0,1034107.0,256772.0,...,806.0,185.0,139.0,74.0,41.0,0,1.164946,0.000000,6.920700,0.000000


In [105]:
# convert date column to float type
df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None)
for i in range(len(df)):
    df['date'][i]=df['date'][i].timestamp()
df['date'] = df['date'].astype('float32')
df.shape

c:\users\jooni\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


(14542, 21)

In [125]:
# select 4 columns for evidence variables
# data = df[['date','state','newCases', 'newDeaths', 'positiveTests','vaccinationsCompleted', 'daily_infection_rate', 'daily_death_rate',
#        'newCases_density', 'newDeaths_density']]
mini_df = df[['date','state','newCases', 'newDeaths']]
mini_df.shape

(14542, 4)

In [126]:
mini_df.tail(10)

,date,state,newCases,newDeaths
14532,1.635552,WY,0.0,0.0
14533,1.635638,WY,0.0,0.0
14534,1.635725,WY,696.0,0.0
14535,1.635811,WY,367.0,69.0
14536,1.635898,WY,414.0,0.0
14537,1.635984,WY,537.0,0.0
14538,1.636070,WY,378.0,0.0
14539,1.636157,WY,0.0,0.0
14540,1.636243,WY,0.0,0.0
14541,1.636330,WY,672.0,0.0


In [127]:
# convert evidence to numpy array and check date type- it should be float
evidence = mini_df.to_numpy()
type(evidence[0][0])
evidence.shape

(14542, 4)

In [129]:
# select mask mandate as a label for categorizing data
mask_as_label = df['mask_mandate'].values
mask_as_label.shape

(14542,)

In [130]:
mask_as_label

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [131]:
# for i in range(len(evidence[0])):
#     np.array(evidence[i][0], dtype=np.float)
#     type(evidence[i][0])

In [132]:
# Separate data into training and testing groups
X_training, X_testing, y_training, y_testing = train_test_split(
    evidence, mask_as_label, test_size=0.4
)

In [133]:
# Create a neural network - meaning one layer after another layer
model = tf.keras.models.Sequential()

In [134]:
# Add a hidden layer with 8 units, with ReLU activation
# Dense layer means each node is densly connected to other layers
model.add(tf.keras.layers.Dense(8, input_shape=(4,), activation="relu"))

In [135]:
# Add output layer with 1 unit, with sigmoid activation
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [136]:
# Train neural network
model.compile(
    optimizer="adam",  # choice of optimization
    loss="binary_crossentropy",  # what type of loss function
    metrics=["accuracy"] # the metrics to analyze for
)

In [140]:
print(X_training.shape, X_testing.shape, y_training.shape, y_testing.shape) 

(8725, 4) (5817, 4) (8725,) (5817,)


In [141]:
X_train=np.asarray(X_train).astype(np.int)
y_train=np.asarray(y_train).astype(np.int)

array([[1.6230239868164062, 'IL', 264.0, 20.0],
       [1.6182719469070435, 'IN', 954.0, 16.0],
       [1.6305409669876099, 'NV', 1136.0, 26.0],
       ...,
       [1.6244930028915405, 'PA', 361.0, 13.0],
       [1.6312320232391357, 'CA', 9861.0, 164.0],
       [1.6339969635009766, 'OH', 4456.0, 202.0]], dtype=object)

In [137]:
# train the model for 20 times
model.fit(X_training, y_training, epochs=10)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [ ]:
# Evaluate and compare with test data:  how well model performs
model.evaluate(X_testing, y_testing, verbose=2)